In [1]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

In [2]:
# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read

# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [3]:
# zHunter stuff
import zhunter.initialize as init
from zhunter.initialize import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb, get_vb_containing, add_crosshair
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem
from zhunter.MainGraphicsWidget import MainGraphicsWidget
from zhunter.data_handler import DataHandler
from zhunter.conversions import convert_flux_to_value, convert_wvlg_to_value, convert_to_bins


In [4]:
# astropalmerio stuff
import astropalmerio.spectra as sp
import astropalmerio.galaxies as gal
import astropalmerio.mc as mc
from astropalmerio.spectra import ergscm2AA
from astropalmerio.spectra import EmissionLine

In [5]:
from zhunter.colors import COLORS
color_style = 'kraken17'
colors = COLORS[color_style]

In [6]:
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
    format="%(asctime)s.%(msecs)03d | %(levelname)-8s | %(funcName)s - %(filename)s:%(lineno)d : %(message)s",

                   )
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

In [7]:
from zhunter.spectrum import OneDSpectrum

# Create Spectrum Object

In [8]:
spec = OneDSpectrum()
# spec = OneDSpectrum(color=colors['specsys'][3])
# spec = OneDSpectrum(color='cyan', width=0.8)

## Load data from a file

In [9]:
fname = '/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/GRB050730_UVES_1D.txt.gz'

spec.load_spectrum_from_file(fname)

# can also load data directly but data must be quantity
# spec.load_spectrum_from_data(
#     wvlg=wvlg,
#     flux=flux,
#     unc=unc,
# )

2023-07-22 15:40:03,406.406 | DEBUG    | read_1D_spectrum - io.py:94 : Read 1D spectrum from file:
/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/GRB050730_UVES_1D.txt.gz
2023-07-22 15:40:03,407.407 | INFO     | read_generic_1D_spectrum - io.py:146 : Attempting to read file:
/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/GRB050730_UVES_1D.txt.gz
2023-07-22 15:40:03,408.408 | DEBUG    | read_generic_1D_spectrum - io.py:165 : Reading with pandas read_csv function
2023-07-22 15:40:03,467.467 | DEBUG    | read_generic_1D_spectrum - io.py:189 : Found the following columns: ['6650.32321600311', '0.0', '0.0020367487']
2023-07-22 15:40:03,468.468 | DEBUG    | find_column_name - io.py:821 : Did not find any name matching ['WAVE', 'AWAV', 'WVLG', 'LAM'] in ['6650.32321600311', '0.0', '0.0020367487']
2023-07-22 15:40:03,469.469 | DEBUG    | find_column_name - io.py:821 : Did not find any name matching ['FLUX', 'SCI'] in ['6650.32321600311', '0.0', '0.0020367487']

In [10]:
spec.info()

2023-07-22 15:40:03,548.548 | INFO     | info - spectrum.py:265 : Properties of <zhunter.spectrum.OneDSpectrum object at 0x18e4743a0>:
{'filename': '/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/GRB050730_UVES_1D.txt.gz',
 'flux_q025': <Quantity -0.158 adu>,
 'flux_q975': <Quantity 0.962 adu>,
 'flux_unit': Unit("adu"),
 'wvlg_max': <Quantity 10426.105 Angstrom>,
 'wvlg_min': <Quantity 6650.323 Angstrom>,
 'wvlg_span': <Quantity 3775.781 Angstrom>,
 'wvlg_step': <Quantity [0.04, 0.04, 0.04, ..., 0.05, 0.05, 0.05] Angstrom>,
 'wvlg_unit': Unit("Angstrom")}
Displayed properties of <zhunter.spectrum.OneDSpectrum object at 0x18e4743a0>:
{'flux_q025_disp': <Quantity -0.158 adu>,
 'flux_q975_disp': <Quantity 0.962 adu>,
 'wvlg_max_disp': <Quantity 10426.105 Angstrom>,
 'wvlg_min_disp': <Quantity 6650.323 Angstrom>,
 'wvlg_span_disp': <Quantity 3775.781 Angstrom>,
 'wvlg_step_disp': <Quantity [0.04, 0.04, 0.04, ..., 0.05, 0.05, 0.05] Angstrom>}


## Change the units
In case the spectrum was read with wrong units

In [11]:
spec.set_flux_unit(u.Unit('1e-18 erg/s/cm2/AA'))
# spec.set_wvlg_unit(u.Unit('nm'))

2023-07-22 15:40:03,560.560 | WARNING  | _showwarning - logger.py:237 : UnitsWarning: '1e-18 erg/s/cm2/AA' contains multiple slashes, which is discouraged by the FITS standard


In [12]:
spec.info()

2023-07-22 15:40:03,576.576 | INFO     | info - spectrum.py:265 : Properties of <zhunter.spectrum.OneDSpectrum object at 0x18e4743a0>:
{'filename': '/Users/palmerio/Code_projects/zHunter/dev/data/test_input_files/GRB050730_UVES_1D.txt.gz',
 'flux_q025': <Quantity -0.158 1e-18 erg / (Angstrom cm2 s)>,
 'flux_q975': <Quantity 0.962 1e-18 erg / (Angstrom cm2 s)>,
 'flux_unit': Unit("1e-18 erg / (Angstrom cm2 s)"),
 'wvlg_max': <Quantity 10426.105 Angstrom>,
 'wvlg_min': <Quantity 6650.323 Angstrom>,
 'wvlg_span': <Quantity 3775.781 Angstrom>,
 'wvlg_step': <Quantity [0.04, 0.04, 0.04, ..., 0.05, 0.05, 0.05] Angstrom>,
 'wvlg_unit': Unit("Angstrom")}
Displayed properties of <zhunter.spectrum.OneDSpectrum object at 0x18e4743a0>:
{'flux_q025_disp': <Quantity -0.158 1e-18 erg / (Angstrom cm2 s)>,
 'flux_q975_disp': <Quantity 0.962 1e-18 erg / (Angstrom cm2 s)>,
 'wvlg_max_disp': <Quantity 10426.105 Angstrom>,
 'wvlg_min_disp': <Quantity 6650.323 Angstrom>,
 'wvlg_span_disp': <Quantity 3775.78

## Visualizing the spectrum

In [13]:
# Classic plotting widget
# fig = pg.PlotWidget()
# vb = fig.plotItem.vb

# or custom plotting widget
from zhunter.OneDGraphicsWidget import OneDGraphicsWidget
fig = OneDGraphicsWidget()
fig.set_up_plot()
vb = fig.ax1D.vb

fig.show()

2023-07-22 15:40:03,591.591 | INFO     | set_up_plot - OneDGraphicsWidget.py:72 : Setting up a new plot called '1D'


In [15]:
vb.addItem(spec.PlotItem)
vb.addItem(spec.PlotItem_unc)

In [16]:
spec.update_plotted_items()

2023-07-22 15:40:21,586.586 | INFO     | update_plotted_items - spectrum.py:145 : Updatind spectrum with bounds: (None, None) and units: (None, None)


## Rebinning

In [17]:
spec.convolve_gaussian(sigma=10)

2023-07-22 15:40:38,587.587 | WARNING  | convolve_gaussian - spectrum.py:314 : Error/uncertainty propagation is not implemented for convolution.
2023-07-22 15:40:38,588.588 | DEBUG    | _set_displayed_data - spectrum.py:198 : Setting the following keys for displaying: ['flux']
2023-07-22 15:40:43,854.854 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: Control, Mouse position: [63,351]
2023-07-22 15:40:43,855.855 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:291 : key press didn't occur in any plot


In [18]:
spec.update_plotted_items()

2023-07-22 15:40:53,143.143 | INFO     | update_plotted_items - spectrum.py:145 : Updatind spectrum with bounds: (None, None) and units: (None, None)


In [19]:
from zhunter.spectral_functions import smooth, convolve_spectrum
from astropy.convolution import Gaussian1DKernel
from scipy.ndimage import gaussian_filter1d

In [18]:
y = gaussian_filter1d(spec.data['flux'].value, sigma=2)

x = convert_to_bins(spec.displayed_data['wvlg'])

pi = pg.PlotCurveItem(
    x,
    y,
    stepMode="center",
    pen=pg.mkPen(
        color='cyan',
        width=1,
    ),
)
vb.addItem(pi)


2023-07-21 14:50:47,722.722 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [563,495]
2023-07-21 14:50:47,995.995 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [563,495]
2023-07-21 14:50:50,716.716 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [777,544]
2023-07-21 14:51:04,196.196 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [793,308]
2023-07-21 14:51:04,627.627 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [793,308]
2023-07-21 14:51:05,196.196 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [793,308]
2023-07-21 14:51:06,403.403 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [793,308]
2023-07-21 14:51:07,252.252 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [793,308]
2023-07-21 14:51:08,351.351 | DEBUG    | keyPres

In [23]:
w, y, u = smooth(
    wvlg=spec.data['wvlg'].value,
    flux=spec.data['flux'].value,
    unc=spec.data['unc'].value,
)

x = convert_to_bins(w)

pi = pg.PlotCurveItem(
    x,
    y,
    stepMode="center",
    pen=pg.mkPen(
        color='purple',
        width=2,
    ),
)
vb.addItem(pi)


2023-07-21 14:55:52,659.659 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [692,433]
2023-07-21 14:55:55,595.595 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [783,433]
2023-07-21 14:55:55,843.843 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [783,433]
2023-07-21 14:55:56,086.086 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [783,433]
2023-07-21 14:55:56,326.326 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [783,433]
2023-07-21 14:55:57,766.766 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [783,433]
2023-07-21 14:55:58,350.350 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [783,433]
2023-07-21 14:55:58,554.554 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [783,433]
2023-07-21 14:55:58,665.665 | DEBUG    | keyPres

2023-07-21 14:57:08,910.910 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:09,090.090 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:09,276.276 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:09,455.455 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:09,628.628 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:09,806.806 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:09,993.993 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:10,418.418 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:10,910.910 | DEBUG    |

2023-07-21 14:57:33,783.783 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:34,003.003 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:34,291.291 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:34,553.553 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:34,818.818 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: D, Mouse position: [1084,258]
2023-07-21 14:57:35,270.270 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: Control, Mouse position: [1084,258]


In [21]:
w, f, u = convolve_spectrum(
    wvlg=spec.data['wvlg'].value,
    flux=spec.data['flux'].value,
    unc=spec.data['unc'].value,
    to_resolution=20000)

2023-07-21 14:51:50,622.622 | INFO     | convolve_spectrum - spectral_functions.py:314 : Spectra convolved!


In [22]:
x = convert_to_bins(w)
y=f

pi = pg.PlotCurveItem(
    x,
    y,
    stepMode="center",
    pen=pg.mkPen(
        color='orange',
        width=3,
    ),
)
vb.addItem(pi)


2023-07-21 14:52:30,041.041 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:30,515.515 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:31,081.081 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:32,693.693 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:32,912.912 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:33,150.150 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:33,880.880 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:34,453.453 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [707,466]
2023-07-21 14:52:34,893.893 | DEBUG    | keyPres

In [44]:
pi.setData(x=convert_to_bins(w), y=f)

2023-07-19 21:35:26,244.244 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:26,830.830 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:26,961.961 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:27,037.037 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:27,106.106 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:27,174.174 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:27,243.243 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:27,312.312 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [945,480]
2023-07-19 21:35:27,381.381 | DEBUG    | keyPres

2023-07-19 21:35:46,638.638 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [920,480]
2023-07-19 21:35:46,811.811 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [920,480]
2023-07-19 21:35:47,266.266 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [920,480]
2023-07-19 21:35:51,277.277 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [827,348]
2023-07-19 21:35:51,507.507 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [827,348]
2023-07-19 21:35:51,668.668 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [827,348]
2023-07-19 21:35:51,833.833 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [827,348]
2023-07-19 21:35:58,946.946 | DEBUG    | keyPressEvent - OneDGraphicsWidget.py:283 : Key: A, Mouse position: [827,348]
2023-07-19 21:35:59,164.164 | DEBUG    | keyPres